In [ ]:
import pandas as pd
import numpy as np
import joblib
import time
import random
import matplotlib.pyplot as plt

brain = joblib.load('brain-2.pkl')


class Personality:
    
    def __init__(self, attribs):
        
        
        self.start = time.time()        
        self.age = [attribs[1]]
        self.gender = [attribs[0]]
        self.mood = [attribs[2]]
        self.health = [attribs[3]]
        self.aggr = [attribs[4]]
        
        self.dx = pd.DataFrame(list(zip(self.gender, self.age, self.mood, self.health, self.aggr)))
        
        
    def update_mood(self):                
        print(self.mood)
    
    
    def update_health(self):
        print(self.health)
    
    
    def update_age(self):        
        self.end = time.time()
        self.age = [round(np.ceil(self.age[0] + (self.end - self.start)), 0)]
        
        
    def get_dataframe(self):
        self.update_age()       
               
        if 0 <= self.age[0] <= 15:
            self.age = ['0-15']
        elif 15 < self.age[0] <= 30:
            self.age = ['15-30']
        elif 30 < self.age[0] <= 50:
            self.age = ['30-50']
        elif 50 < self.age[0] <= 70:
            self.age = ['50-70']
        elif self.age[0] > 70:
            self.age = ['70+']
            
            
        self.dx = pd.DataFrame(list(zip(self.gender, self.age, self.mood, self.health, self.aggr)),
                              columns=['gender', 'age', 'mood', 'health', 'aggression'])
        return self.dx
        

def preprocess(df, unseen=True):
    
    df.mood = df.mood.map({'happy': 1, 'unhappy': 0})
    df.health = df.health.map({'healthy': 1, 'unhealthy': 0})    
    df.gender = df.gender.map({'m': 1, 'f': 0})
    df.age = df.age.map({'0-15': 0, '15-30': 1, '30-50': 2, '50-70': 3, '70+': 4})
    df.aggression = df.aggression.map({'high': 1, 'low': 0})
    
    if not unseen:                                                  # If unseen is false
        df.action = df.action.map({'MRD': 0, 'R': 1})
        X = df.drop('action', axis=1)
        y = df.action
        y = np.asarray(y)
    
        return df, X, y
    else:
        return df

    
def preprocess2(dt):
    
    age = ''
    
    if 0 <= dt.age[0] <= 15:
        age = '0-15'
    elif 15 < dt.age[0] <= 30:
        age = '15-30'
    elif 30 < dt.age[0] <= 50:
        age = '30-50'
    elif 50 < dt.age[0] <= 70:
        age = '50-70'
    elif dt.age[0] > 70:
        age = '70+'
   
   
    
    df = pd.DataFrame(list(zip(dt.gender, [age], dt.mood, dt.health, dt.aggression)),
                              columns=['gender', 'age', 'mood', 'health', 'aggression'])
    
    df.age = df.age.map({'0-15': 0, '15-30': 1, '30-50': 2, '50-70': 3, '70+': 4})
    df.mood = df.mood.map({'happy': 1, 'unhappy': 0})
    df.health = df.health.map({'healthy': 1, 'unhealthy': 0})    
    df.gender = df.gender.map({'m': 1, 'f': 0})
    df.aggression = df.aggression.map({'high': 1, 'low': 0})
    
    return df    
    
    
def postprocess(df):
    
    df.action = df.action.map({0: 'MRD', 1: 'R'})
    df.mood = df.mood.map({1: 'happy', 0: 'unhappy'})
    df.health = df.health.map({1:'healthy', 0: 'unhealthy'})    
    df.gender = df.gender.map({1: 'm', 0: 'f'})
    df.age = df.age.map({0: '0-15', 1: '15-30', 2: '30-50', 3: '50-70', 4: '70+'})
    df.aggression = df.aggression.map({1: 'high', 0: 'low'})
        
    return df    


def think(df):
    
    df = preprocess(df)
    df.age = df.age.replace(np.NaN, 1)
    df['action'] = brain.predict(df)
    df = postprocess(df)
    return df


def think2(df):
    df = preprocess2(df)
    df['action'] = brain.predict(df)
    df = postprocess(df)
    return df

In [ ]:
class Interact:
    
    def __init__(self, df1, df2):
        
        self.df1 = df1
        self.df2 = df2
        self.genmatch = None
        self.moodmatch = None
        self.aggrmatch = None
        self.actionmatch = None
        self.healthmatch = None
        self.start = time.time()
        
        
    
    def check_gender(self):
        if self.df1.gender[0] == self.df2.gender[0]:
            self.genmatch = True
        else:
            self.genmatch = False
            
    
    def check_mood(self):
        if self.df1.mood[0] == self.df2.mood[0]:
            self.moodmatch = True
        else:
            self.moodmatch = False
            
    
    def check_aggression(self):
        if self.df1.aggression[0] == self.df2.aggression[0]:
            self.aggrmatch = True
        else:
            self.aggrmatch = False    
     

    def check_health(self):
        if self.df1.health[0] == self.df2.health[0]:
            self.healthmatch = True
        else:
            self.healthmatch = False
            
            
    def check_action(self):
        if self.df1.action[0] == self.df2.action[0]:
            self.actionmatch = True
        else:
            self.actionmatch = False

            
    def run_interaction(self, show=False):

        self.murders = []
        self.bastards = []
        self.legits = []
        self.check_action()
        self.check_gender()
        self.check_aggression()
        self.kill_flag = False
        self.rd = pd.DataFrame()

        gender = [np.random.choice(['m', 'f'])]
        mood = [np.random.choice([self.df1.mood[0], self.df2.mood[0]])]
        end = time.time()
        age = [np.random.randint(0, 85)]
        health = [np.random.choice([self.df1.health[0], self.df2.health[0]])]
        aggr = [np.random.choice([self.df1.aggression[0], self.df2.aggression[0]])]

        # RULES

        if not self.genmatch and self.aggrmatch and self.actionmatch:
            if self.df1.action[0] == 'R':
                rx = pd.DataFrame(list(zip(gender, age, mood, health, aggr)),
                                  columns=['gender', 'age', 'mood', 'health', 'aggression'])

                print(f'\033[1;32mInteraction resulted in Mating.\033[0m')
                self.legits.append(1)
                return rx
        
        if not self.genmatch and not self.aggrmatch and not self.actionmatch:
            if self.df1.gender[0] == 'm' and self.df2.gender[0] == 'f' and self.df1.mood[0] == 'unhappy':
                if self.df1.aggression[0] == 'high' and self.df1.action[0] == 'R' and not self.df2.age[0] == '70+':
                    rx = pd.DataFrame(list(zip(gender, age, mood, health, aggr)),
                                  columns=['gender', 'age', 'mood', 'health', 'aggression'])
                    print(f'\033[0;31mInteraction resulted in illegitimate child.\033[0m')
                    self.bastards.append(1)
                    return rx
            elif self.df1.gender[0] == 'f' and self.df2.gender[0] == 'm' and self.df2.mood[0] == 'unhappy':
                if self.df2.aggression[0] == 'high' and self.df2.action[0] == 'R' and not self.df1.age[0] == '70+':
                    rx = pd.DataFrame(list(zip(gender, age, mood, health, aggr)),
                                  columns=['gender', 'age', 'mood', 'health', 'aggression'])
                    print(f'\033[0;31mInteraction resulted in illegitimate child.\033[0m')
                    self.bastards.append(1)
                    return rx
                
        if self.genmatch and not self.aggrmatch and not self.actionmatch:
            if self.df1.gender[0] == 'm' and self.df1.action[0] == 'R' and self.df1.aggression[0] == 'high':
                if self.df2.mood[0] == 'unhappy' and not self.df2.age[0] == '70+':
                    print(f'\033[1;31mInteraction resulted in Murder.\033[0m')                    
                    self.murders.append(1)
                    self.kill_flag = True
                    indexparam = self.df2[(self.df2['aggression'] == 'low')].index                    
                    ind = indexparam[0]                    
                    self.rd = self.df2[self.df2.index == ind]      # rd has to be removed from main holding file

#### INFERENCE 1 - Creation of Founder Generation (GEN-0)

In [ ]:
pop = 15

In [ ]:
DEBUG = False
count = 1
while count <= pop:
    pdf = pd.read_csv('result_gen0.csv')
    age = np.random.randint(0, 85)
    gender = np.random.choice(['m', 'f'])
    mood = np.random.choice(['happy', 'unhappy'])
    health = np.random.choice(['healthy', 'unhealthy'])
    aggr = np.random.choice(['high', 'low'])

    attribs = [gender, age, mood, health, aggr]

    P1 = Personality(attribs)
    df1 = P1.get_dataframe()
    df1 = think(df1)

    age = np.random.randint(0, 85)
    gender = np.random.choice(['m', 'f'])
    mood = np.random.choice(['happy', 'unhappy'])
    health = np.random.choice(['healthy', 'unhealthy'])
    aggr = np.random.choice(['high', 'low'])

    attribs = [gender, age, mood, health, aggr]

    P2 = Personality(attribs)
    df2 = P2.get_dataframe()
    df2 = think(df2)
    
    if DEBUG: display(df1)
    if DEBUG: display(df2)
    
    I = Interact(df1, df2)
    res = I.run_interaction(show=False)
    pdf1 = pdf.append(res)
    if not DEBUG: pdf1.to_csv('result_gen0.csv', index=None)
    count += 1

In [ ]:
print(f'Total Founder Generation (Gen0) = {pdf1.shape[0]}')      
print(f'pdf.shape: {pdf.shape}')
print(pdf.mood.value_counts())
print(pdf.health.value_counts())
print(pdf.aggression.value_counts())
print(pdf.gender.value_counts())

In [ ]:
def run_simulation(gen_file, epochs, show=False, DEBUG=False):
    
    start = time.time()
    
    pdf = pd.DataFrame()
    murders = []
    bastards = []
    legits = []
    count = 0
    while count <= epochs:
        
        ppd = pd.read_csv(gen_file)

        dfa = ppd.sample().reset_index(drop=True)
        dfb = ppd.sample().reset_index(drop=True)

        dfa = think2(dfa)
        dfb = think2(dfb)

        if DEBUG: display(dfa)
        if DEBUG: display(dfb)

        I = Interact(dfa, dfb)
        rdf = I.run_interaction(show=show)
        
        murders.append(I.murders)
        bastards.append(I.bastards)
        legits.append(I.legits)
        if DEBUG: display(rdf)        
        
        pdf = pdf.append(rdf)
        
        if I.kill_flag:
            try:
                a_index = pdf.set_index(['gender']).index
                b_index = I.rd.set_index(['gender']).index

                mask = ~a_index.isin(b_index)
                pdf = pdf.loc[mask]
                pdf = pdf.reset_index(drop=True)
            except:
                print(f'Excepted in Kill Flag')
        count += 1
    
    murders = list(filter(None, murders))
    bastards = list(filter(None, bastards))
    legits = list(filter(None, legits))
    
    return pdf, murders, bastards, legits

In [ ]:
pdf, murders, bastards, legits = run_simulation('test1.csv', epochs=100)

In [ ]:
def start_master_simulation(GEN):
    counter = 0
    murders = []
    bastards = []
    legits = []
    pop = []
    mood = []
    aggr = []
    health = []
    emul = 1.01
    
    edf = pd.DataFrame()
    pf = pd.read_csv('result_gen0.csv')    
    pf.to_csv('result_genx.csv', index=None)     # Creating copy of gen0 file to propagate
    
    print(f'Starting Population Size: {pf.shape[0]}')
    
    while counter <= GEN:
        
        pf = pd.read_csv('result_genx.csv')
        EPOCHS = int(np.floor(pf.shape[0] * emul))
        print(f'EPOCHS this Generation: {EPOCHS}')
        print(f'Population Size: {pf.shape[0]}')
        
        dx, m, b, l = run_simulation('result_gen0.csv',epochs=EPOCHS)
        print(f'Rows received from interactions: {dx.shape[0]}')
        pf = pf.append(dx)        
        pf.to_csv('result_genx.csv', index=None)
        
        murders.append(len(m))
        bastards.append(len(b))
        legits.append(len(l))
        pop.append(pf.shape[0])

        if pf.mood.value_counts().index[0] == 'happy': 
            mood.append(pf.mood.value_counts()[0])
        elif pf.aggression.value_counts().index[0] == 'low': 
            mood.append(pf.mood.value_counts()[1])

        if pf.health.value_counts().index[0] == 'healthy': 
            health.append(pf.health.value_counts()[0])
        elif pf.health.value_counts().index[0] == 'unhealthy': 
            health.append(pf.health.value_counts()[1])

        if pf.aggression.value_counts().index[0] == 'high': 
            aggr.append(pf.aggression.value_counts()[0])
        elif pf.aggression.value_counts().index[0] == 'low': 
            aggr.append(pf.aggression.value_counts()[1])
        
        print(pf.mood.value_counts())
        print(pf.health.value_counts())
        print(pf.aggression.value_counts())
        
        print(f'+--------------+ END OF GENERATION {counter} +---------------+')
        counter += 1
        
    return pf, pop, mood, aggr, health, murders, bastards, legits
    

### INFERENCE 2 

In [ ]:
GEN = 24

In [ ]:
dx, pop, mood, aggr, health, murders, bastards, legits = start_master_simulation(GEN)

In [ ]:
print(f'\nGenerations: {GEN}')
print(f'\nMurders: {murders}')
print(f'Bastards: {bastards}')
print(f'Legits: {legits}')
print(f'Pop: {pop}')
print(f'Mood: {mood}')
print(f'Health: {health}')
print(f'Aggression: {aggr}\n')

In [ ]:
def get_percentage(lst1, lst2, label):
    atd = {}
    for x, y in enumerate(zip(lst1, lst2)):
        res = round(y[0]/y[1] * 100, 1)
        atd[f"{label}{x}"] = f'{res}%'
    return atd



def get_pc_values(dic):
    hol = []
    for k, v in dic.items():
        hol.append(v)
    hol = [x.replace('%', '') for x in hol]
    hol = [float(x) for x in hol]
    return hol



def plot_graph(lst, GEN, colorsl, ylabel='', title='', legendlabels='', figsize=(12,7)):
    
    '''Args:
    lst - <list>
    GEN - <int>
    colorsl - <list>
    
    Optional:
    ylabel - <str>
    title - <str>
    figsize - <tuple>
    
    Ex. 
    
    colorsl = ['r', 'b', 'g']
    lst = ['mood', 'hap', 'aggr']
    GEN = 20
    title = 'Plot'
    ylabel = 'Mock'
    figsize = (10, 4)
    
    plot_graph(lst, GEN, colorsl, ylabel=ylabel, title=title, figsize=figsize)
    '''
    
    
    x = list(np.arange(GEN+1))
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=figsize)
    ax.set_xlabel('Generations', fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.set_title(title, fontsize=15)
    
    for z1, z2, z3 in zip(lst, colorsl, legendlabels):
        plt.plot(x, z1, color=z2, label=z3, axes=ax)    
    
        plt.legend()
    plt.show()


In [ ]:
hap = get_percentage(mood, pop, 'happiness')
hel = get_percentage(health, pop, 'healthiness')
agg = get_percentage(aggr, pop, 'aggression')
    
hapl = get_pc_values(hap) 
hell = get_pc_values(hel) 
aggl = get_pc_values(agg) 

In [ ]:
lst = [hapl, hell, aggl]
lst2 = [murders, bastards, legits]
lst3 = [health, aggr, pop]

colors = ['limegreen', 'orchid', 'orange']
colors2 = ['darksalmon', 'slateblue', 'forestgreen']
colors3 = ['green', 'dodgerblue', 'red', 'purple']

legendlabels = ['Happiness %', 'Healthiness %', 'Aggression %']
legendlabels2 = ['Murders', 'Bastards', 'Legits']
legendlabels3 = ['Happiness', 'Health', 'Aggression', 'Population']

plot_graph(lst, GEN, colors, ylabel='Percentage', title='Percent Stats', 
           legendlabels=legendlabels, figsize=(12,7))

plot_graph(lst2, GEN, colors2, ylabel='Population Size', title='Crime Stats over Generations',
           legendlabels=legendlabels2, figsize=(12,7))

plot_graph(lst3, GEN, colors3, ylabel='Percentage', title='Population Stats over Generations',
           legendlabels=legendlabels3, figsize=(12,7))



In [ ]:
lst = [hell, aggl]
lst2 = [murders, bastards, legits]
lst3 = [health, aggr, pop]

colors = ['orchid', 'orange']
colors2 = ['darksalmon', 'slateblue', 'forestgreen']
colors3 = ['orchid', 'red', 'purple']

legendlabels = ['Healthiness %', 'Aggression %']
legendlabels2 = ['Murders', 'Bastards', 'Legits']
legendlabels3 = ['Health', 'Aggression', 'Population']

plot_graph(lst, GEN, colors, ylabel='Percentage', title='Percent Stats', 
           legendlabels=legendlabels, figsize=(12,7))

plot_graph(lst2, GEN, colors2, ylabel='Population Size', title='Crime Stats over Generations',
           legendlabels=legendlabels2, figsize=(12,7))

plot_graph(lst3, GEN, colors3, ylabel='Percentage', title='Population Stats over Generations',
           legendlabels=legendlabels3, figsize=(12,7))

In [ ]:
print('\033[1;32m--END\033[0m')

In [ ]:
print('Now Pushed through Pythonified Gitpush commands. Yay!!!')